In [1]:
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import os

# CSV 파일 불러오기
data = pd.read_csv('emotion.csv')

# 음성 데이터 특징 추출 함수
def extract_features(wav_id):
    file_path = f'sound/{wav_id}.wav'  # 수정된 파일 경로
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
        tonnetz = librosa.feature.tonnetz(y=audio, sr=sample_rate)
        features = np.mean(np.concatenate([mfccs, chroma, mel, contrast, tonnetz], axis=0), axis=1)
    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, {e}")
        return None
    return features

# 병렬 처리를 사용하여 모든 음성 파일에 대해 특징 추출 수행
features = Parallel(n_jobs=-1)(delayed(extract_features)(wav_id) for wav_id in tqdm(data['wav_id'], desc="특징 추출"))

# None 값을 제거하고 numpy 배열로 변환
features = np.array([x for x in features if x is not None])

# features 배열의 길이 확인
print(f"Features 배열의 길이: {len(features)}")

# 라벨 준비
y = data['1번 감정'].values

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

# 모델 학습
print("모델 학습 중...")
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
print("모델 평가 중...")
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

# 정확도 0.45


c:\Users\jhy97\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
특징 추출: 100%|██████████| 10011/10011 [05:23<00:00, 30.98it/s]


Features 배열의 길이: 10011
모델 학습 중...
모델 평가 중...
Accuracy: 0.4588117823265102


In [10]:
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import os

# CSV 파일 불러오기
data = pd.read_csv('emotion.csv')

data['성별'] = data['성별'].map({'female': 0, 'male': 1})

# LabelEncoder 인스턴스 생성
le = LabelEncoder()

# '상황', '1번 감정' ~ '5번 감정' 열에 대해 Label encoding 적용
for col in ['상황', '1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정']:
    data[col] = le.fit_transform(data[col])



# 음성 데이터 특징 추출 함수
def extract_features(wav_id):
    file_path = f'sound/{wav_id}.wav'
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
        tonnetz = librosa.feature.tonnetz(y=audio, sr=sample_rate)
        features = np.mean(np.concatenate([mfccs, chroma, mel, contrast, tonnetz], axis=0), axis=1)
    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, {e}")
        return None
    return features

# 병렬 처리를 사용하여 모든 음성 파일에 대해 특징 추출 수행
features = Parallel(n_jobs=-1)(delayed(extract_features)(wav_id) for wav_id in tqdm(data['wav_id'], desc="특징 추출"))

# None 값을 제거하고 numpy 배열로 변환
features = np.array([x for x in features if x is not None])

# 데이터셋 분할
X = features  # 앞서 추출한 특징(features) 사용
y = data[['상황', '1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정', '1번 감정세기', '2번 감정세기', '3번 감정세기', '4번감정세기', '5번 감정세기', '나이', '성별']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=200, random_state=42))
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')


In [14]:
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import os

# CSV 파일 불러오기
data = pd.read_csv('emotion.csv')

data['성별'] = data['성별'].map({'female': 0, 'male': 1})

# LabelEncoder 인스턴스 생성
le = LabelEncoder()

# '상황', '1번 감정' ~ '5번 감정' 열에 대해 Label encoding 적용
for col in ['상황', '1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정']:
    data[col] = le.fit_transform(data[col])

# 음성 데이터 특징 추출 함수
def extract_features(wav_id):
    file_path = f'sound/{wav_id}.wav'
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
        tonnetz = librosa.feature.tonnetz(y=audio, sr=sample_rate)
        features = np.mean(np.concatenate([mfccs, chroma, mel, contrast, tonnetz], axis=0), axis=1)
    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, {e}")
        return None
    return features

# 병렬 처리를 사용하여 모든 음성 파일에 대해 특징 추출 수행
features = Parallel(n_jobs=-1)(delayed(extract_features)(wav_id) for wav_id in tqdm(data['wav_id'], desc="특징 추출"))

# None 값을 제거하고 numpy 배열로 변환
features = np.array([x for x in features if x is not None])

# None 값이 제거된 행에 해당하는 y 값 선택
valid_rows = [i for i, x in enumerate(features) if x is not None]
y = data.iloc[valid_rows][['상황', '1번 감정', '2번 감정', '3번 감정', '4번 감정', '5번 감정', '1번 감정세기', '2번 감정세기', '3번 감정세기', '4번감정세기', '5번 감정세기', '나이', '성별']].values

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

# 모델 학습
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=200, random_state=42))
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)

# 예측된 결과와 실제 값을 비교하여 각 레이블에 대한 정확도를 계산
for i in range(y_test.shape[1]):
    label_accuracy = accuracy_score(y_test[:, i], y_pred[:, i])
    print(f'레이블 {i} 정확도: {label_accuracy}')



















































































































































































































































































































































































































































































































































































































































특징 추출: 100%|██████████| 10011/10011 [04:12<00:00, 39.60it/s]


레이블 0 정확도: 0.5841238142785822
레이블 1 정확도: 0.4588117823265102
레이블 2 정확도: 0.6010983524712931
레이블 3 정확도: 0.7908137793310035
레이블 4 정확도: 0.6160758861707439
레이블 5 정확도: 0.5896155766350474
레이블 6 정확도: 0.5551672491263105
레이블 7 정확도: 0.6245631552670994
레이블 8 정확도: 0.800798801797304
레이블 9 정확도: 0.8831752371442836
레이블 10 정확도: 0.7888167748377434
레이블 11 정확도: 0.9745381927109336
레이블 12 정확도: 0.9950074887668497


In [ ]:
import matplotlib.pyplot as plt

# 첫 번째 타겟에 대한 특징 중요도 추출
feature_importances = model.estimators_[0].feature_importances_

# 특징 중요도 시각화
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importances)), feature_importances)
plt.xlabel('Feature Index')
plt.ylabel('Importance')
plt.title('Feature Importance for the First Target')
plt.show()


In [1]:
# 대화 데이터 구조를 정의하는 클래스
class Conversation:
    def __init__(self, gender, keywords, health_condition, emotion_main, emotion_sub, sentences):
        self.gender = gender
        self.keywords = keywords
        self.health_condition = health_condition
        self.emotion_main = emotion_main
        self.emotion_sub = emotion_sub
        self.sentences = sentences  # 사람과 시스템의 문장 쌍을 리스트로 저장

# 대화 내용을 분석하는 함수
def analyze_conversation(conversation):
    print("성별:", conversation.gender)
    print("상황 키워드:", conversation.keywords)
    print("신체 질환:", conversation.health_condition)
    print("감정 대분류:", conversation.emotion_main)
    print("감정 소분류:", conversation.emotion_sub)
    print("대화 내용:")
    for i, (person_sentence, system_sentence) in enumerate(conversation.sentences, 1):
        print(f"  대화 {i} - 사람: {person_sentence}")
        print(f"         시스템: {system_sentence}")

# 샘플 대화 데이터 생성
sample_conversation = Conversation(
    gender="여성",
    keywords="진로,취업,직장",
    health_condition="해당없음",
    emotion_main="분노",
    emotion_sub="노여워하는",
    sentences=[
        ("일은 왜 해도 해도 끝이 없을까? 화가 난다.", "많이 힘드시겠어요. 주위에 의논할 상대가 있나요?"),
        ("그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.", "혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.")
    ]
)

# 샘플 대화 분석 실행
analyze_conversation(sample_conversation)


성별: 여성
상황 키워드: 진로,취업,직장
신체 질환: 해당없음
감정 대분류: 분노
감정 소분류: 노여워하는
대화 내용:
  대화 1 - 사람: 일은 왜 해도 해도 끝이 없을까? 화가 난다.
         시스템: 많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
  대화 2 - 사람: 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.
         시스템: 혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.
